### In this notebook:
I perform random search with xgboost model to find hyperparameters. I use feature matrix with 10% sampled data from original dataset. Feature matrix has selected variables after feature selection. See notebook `feature_selection`.

### About project:
Data is from Kaggle competiotion [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk). 

I implement an automated feature engineering approach with an open-source library [Featuretools](https://www.featuretools.com/). 


In [67]:
import xgboost as xgb 

In [119]:
import numpy as np
import pandas as pd
import time

In [69]:
import matplotlib.pyplot as plt
%matplotlib inline

In [80]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV

In [4]:
fm_sample = pd.read_csv('../data/fm_train_sample_trans.csv')

In [72]:
id_col = 'SK_ID_CURR'
target_col = 'TARGET'

predictors = [x for x in fm_sample.columns if x not in [id_col, target_col]]

In [73]:
#X_train, X_test, y_train, y_test = train_test_split(fm_sample[predictors], fm_sample[target_col], test_size=0.2, random_state=42)

In [74]:
id_col = 'SK_ID_CURR'
target_col = 'TARGET'

In [75]:
predictors = [x for x in fm_sample.columns if x not in [id_col, target_col]]

In [76]:
X_train = fm_sample[predictors]
y_train = fm_sample[target_col]

In [50]:
#dtrain = xgb.DMatrix(data=X_train.values, label=y_train.values, feature_names=predictors)
#dtest = xgb.DMatrix(data=X_test.values, label=y_test.values, feature_names=predictors)

In [200]:
clf = xgb.XGBClassifier(objective='binary:logistic')

In [202]:
param_dist = {
    'max_depth': range(3,10),
    'learning_rate': np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 1000),
    'n_estimators': range(10, 60, 20)
    'silent':1,
    'objective': 'binary:logistic',
    'gamma': np.linspace(0, 0.4, 6),
    'min_child_weight': range(1,6,2),
    'subsample': np.linspace(0.5, 1, 101),
    'colsample_bytree': np.linspace(0.6, 1, 11),
    'colsample_bylevel': np.linspace(0.6, 1, 11),
    'reg_alpha': np.linspace(0, 1),
    'reg_lambda': np.linspace(0, 1)
    
    #'n_estimators': range(100, 1100, 100),
}

In [203]:
y_train.count()

30751

In [204]:
X_train.shape

(30751, 1036)

In [205]:
n_iter_search = 3

In [206]:
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=n_iter_search, cv=5, scoring='roc_auc' )

In [1]:
start = time.time()


random_search.fit(X=X_train.values, y=y_train.values)


end = time.time()
print('Computation took {} sec.'.format(round(end - start), 1))